## Install Prerequisities

In [1]:
## basics
!sudo apt-get install python3 python-dev python3-dev \
     build-essential libssl-dev libffi-dev \
     libxml2-dev libxslt1-dev zlib1g-dev \
     python-pip
!sudo apt install libosmesa6-dev libgl1-mesa-glx libglfw3
## patchelf
!sudo apt-get update
!sudo apt install patchelf
## mujoco
!wget https://www.roboti.us/download/mjpro150_linux.zip
!unzip mjpro150_linux.zip 
!mkdir ~/.mujoco
!mv mjpro150 ~/.mujoco/

import os
os.environ['LD_LIBRARY_PATH'] += ":/root/.mujoco/mjpro150/bin"
!echo $LD_LIBRARY_PATH

# !wget https://raw.githubusercontent.com/ShangtongZhang/DeepRL/DAC/requirements.txt
# !pip install -r requirements.txt
!wget https://github.com/openai/mujoco-py/archive/1.50.1.0.zip
!unzip 1.50.1.0.zip
%cd mujoco-py-1.50.1.0/
!pip install -e .
%cd /content/

Reading package lists... Done
Building dependency tree       
Reading state information... Done
build-essential is already the newest version (12.4ubuntu1).
python-dev is already the newest version (2.7.15~rc1-1).
zlib1g-dev is already the newest version (1:1.2.11.dfsg-0ubuntu2).
zlib1g-dev set to manually installed.
libssl-dev is already the newest version (1.1.1-1ubuntu2.1~18.04.4).
libxml2-dev is already the newest version (2.9.4+dfsg1-6.1ubuntu1.2).
python3 is already the newest version (3.6.7-1~18.04).
python3-dev is already the newest version (3.6.7-1~18.04).
The following additional packages will be installed:
  libpython-all-dev python-all python-all-dev python-asn1crypto
  python-cffi-backend python-crypto python-cryptography python-dbus
  python-enum34 python-gi python-idna python-ipaddress python-keyring
  python-keyrings.alt python-pip-whl python-pkg-resources python-secretstorage
  python-setuptools python-six python-wheel python-xdg
Suggested packages:
  python-crypto-doc

In [0]:
!echo $LD_LIBRARY_PATH
%cd mujoco-py-1.50.1.0/
import mujoco_py
import os
mj_path, _ = mujoco_py.utils.discover_mujoco()
xml_path = os.path.join(mj_path, 'model', 'humanoid.xml')
model = mujoco_py.load_model_from_path(xml_path)
sim = mujoco_py.MjSim(model)

print(sim.data.qpos)
# [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

sim.step()
print(sim.data.qpos)

/usr/lib64-nvidia:/root/.mujoco/mjpro150/bin
/content/mujoco-py-1.50.1.0
Compiling /content/mujoco-py-1.50.1.0/mujoco_py/cymj.pyx because it changed.
[1/1] Cythonizing /content/mujoco-py-1.50.1.0/mujoco_py/cymj.pyx


running build_ext
building 'mujoco_py.cymj' extension
creating /content/mujoco-py-1.50.1.0/mujoco_py/generated/_pyxbld_LinuxCPUExtensionBuilder
creating /content/mujoco-py-1.50.1.0/mujoco_py/generated/_pyxbld_LinuxCPUExtensionBuilder/temp.linux-x86_64-3.6
creating /content/mujoco-py-1.50.1.0/mujoco_py/generated/_pyxbld_LinuxCPUExtensionBuilder/temp.linux-x86_64-3.6/content
creating /content/mujoco-py-1.50.1.0/mujoco_py/generated/_pyxbld_LinuxCPUExtensionBuilder/temp.linux-x86_64-3.6/content/mujoco-py-1.50.1.0
creating /content/mujoco-py-1.50.1.0/mujoco_py/generated/_pyxbld_LinuxCPUExtensionBuilder/temp.linux-x86_64-3.6/content/mujoco-py-1.50.1.0/mujoco_py
creating /content/mujoco-py-1.50.1.0/mujoco_py/generated/_pyxbld_LinuxCPUExtensionBuilder/temp.linux-x86_64-3.6/content/mujoco-py-1.50.1.0/mujoco_py/gl
x86_64-linux-gnu-gcc -pthread -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -Imujoco_py -I/content/mu

In [12]:
# !cp /root/.mujoco/mjkey.txt /root/.mujoco/mujoco200
# os.environ['MUJOCO_PY_MJKEY_PATH'] = "/root/.mujoco/mujoco200"
# !/root/.mujoco/mujoco200/bin/simulate /root/.mujoco/mujoco200/bin/model/humanoid.xml
!echo $MUJOCO_PY_MJKEY_PATH

/root/.mujoco/mujoco200


In [13]:
!sudo osmesa

sudo: osmesa: command not found


## Import packages

In [0]:
import math
import random

import gym
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.distributions import Normal

from IPython.display import clear_output
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
use_cuda = torch.cuda.is_available()
device   = torch.device("cuda" if use_cuda else "cpu")

In [4]:
!git clone https://github.com/DAC-Prime/supreme-waffle.git

Cloning into 'supreme-waffle'...
remote: Enumerating objects: 45, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 45 (delta 15), reused 29 (delta 9), pack-reused 0
Unpacking objects: 100% (45/45), done.


## Create Environments

In [0]:
from env import 

env = gym.make()

## Neural Network

In [0]:
def init_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.normal_(m.weight, mean=0., std=0.1)
        nn.init.constant_(m.bias, 0.1)   

class ActorCritic(nn.Module):
    def __init__(self, num_inputs, num_outputs, hidden_size, std=0.0):
        super(ActorCritic, self).__init__()
        
        self.critic = nn.Sequential(
            nn.Linear(num_inputs, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, 1)
        )
        
        self.actor = nn.Sequential(
            nn.Linear(num_inputs, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, num_outputs),
        )
        self.log_std = nn.Parameter(torch.ones(1, num_outputs) * std)
        
        self.apply(init_weights)
        
    def forward(self, x):
        value = self.critic(x)
        mu    = self.actor(x)
        std   = self.log_std.exp().expand_as(mu)
        dist  = Normal(mu, std)
        return dist, value

def plot(frame_idx, rewards):
    clear_output(True)
    plt.figure(figsize=(20,5))
    plt.subplot(131)
    plt.title('frame %s. reward: %s' % (frame_idx, rewards[-1]))
    plt.plot(rewards)
    plt.show()
    
def test_env(vis=False):
    state = env.reset()
    if vis: env.render()
    done = False
    total_reward = 0
    while not done:
        state = torch.FloatTensor(state).unsqueeze(0).to(device)
        dist, _ = model(state)
        next_state, reward, done, _ = env.step(dist.sample().cpu().numpy()[0])
        state = next_state
        if vis: env.render()
        total_reward += reward
    return total_reward

def compute_gae(next_value, rewards, masks, values, gamma=0.99, tau=0.95):
    values = values + [next_value]
    gae = 0
    returns = []
    for step in reversed(range(len(rewards))):
        delta = rewards[step] + gamma * values[step + 1] * masks[step] - values[step]
        gae = delta + gamma * tau * masks[step] * gae
        returns.insert(0, gae + values[step])
    return returns

## PPO Algorithm

In [0]:
def ppo_iter(mini_batch_size, states, actions, log_probs, returns, advantage):
    batch_size = states.size(0)
    ids = np.random.permutation(batch_size)
    ids = np.split(ids[:batch_size // mini_batch_size * mini_batch_size], batch_size // mini_batch_size)
    for i in range(len(ids)):
        yield states[ids[i], :], actions[ids[i], :], log_probs[ids[i], :], returns[ids[i], :], advantage[ids[i], :]      

def ppo_update(ppo_epochs, mini_batch_size, states, actions, log_probs, returns, advantages, clip_param=0.2):
    for _ in range(ppo_epochs):
        for state, action, old_log_probs, return_, advantage in ppo_iter(mini_batch_size, states, actions, log_probs, returns, advantages):
            dist, value = model(state)
            entropy = dist.entropy().mean()
            new_log_probs = dist.log_prob(action)

            ratio = (new_log_probs - old_log_probs).exp()
            surr1 = ratio * advantage
            surr2 = torch.clamp(ratio, 1.0 - clip_param, 1.0 + clip_param) * advantage

            actor_loss  = - torch.min(surr1, surr2).mean()
            critic_loss = (return_ - value).pow(2).mean()

            loss = 0.5 * critic_loss + actor_loss - 0.001 * entropy

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

## Training Set Up

In [0]:
num_inputs  = envs.observation_space.shape[0]
num_outputs = envs.action_space.shape[0]

#Hyper params:
threshold_reward = -200

hidden_size      = 32
lr               = 1e-3
num_steps        = 128
mini_batch_size  = 256
ppo_epochs       = 30

model = ActorCritic(num_inputs, num_outputs, hidden_size).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)

max_frames = 15000
frame_idx  = 0
test_rewards = []

state = envs.reset()
early_stop = False

while frame_idx < max_frames and not early_stop:

    log_probs = []
    values    = []
    states    = []
    actions   = []
    rewards   = []
    masks     = []
    entropy = 0

    for _ in range(num_steps):
        state = torch.FloatTensor(state).to(device)
        dist, value = model(state)

        action = dist.sample()
        next_state, reward, done, _ = envs.step(action.cpu().numpy())

        log_prob = dist.log_prob(action)
        entropy += dist.entropy().mean()
        
        log_probs.append(log_prob)
        values.append(value)
        rewards.append(torch.FloatTensor(reward).unsqueeze(1).to(device))
        masks.append(torch.FloatTensor(1 - done).unsqueeze(1).to(device))
        
        states.append(state)
        actions.append(action)
        
        state = next_state
        frame_idx += 1
        
        if frame_idx % 1000 == 0:
            test_reward = np.mean([test_env() for _ in range(10)])
            test_rewards.append(test_reward)
            plot(frame_idx, test_rewards)
            if test_reward > threshold_reward: early_stop = True
            

    next_state = torch.FloatTensor(next_state).to(device)
    _, next_value = model(next_state)
    returns = compute_gae(next_value, rewards, masks, values)

    returns   = torch.cat(returns).detach()
    log_probs = torch.cat(log_probs).detach()
    values    = torch.cat(values).detach()
    states    = torch.cat(states)
    actions   = torch.cat(actions)
    advantage = returns - values
    
    ppo_update(ppo_epochs, mini_batch_size, states, actions, log_probs, returns, advantage)

# Using Stable Baselines Package

In [6]:
!sudo apt-get update && sudo apt-get install cmake libopenmpi-dev python3-dev zlib1g-dev
!pip install stable-baselines

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease
Hit:12 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree       
Reading state information... Done
zlib1g-

In [51]:
import gym

from stable_baselines.common.policies import MlpPolicy
from stable_baselines.common.vec_env import DummyVecEnv, VecNormalize
from stable_baselines import PPO2

env = DummyVecEnv([lambda: gym.make("Reacher-v2")])
# Automatically normalize the input features
env = VecNormalize(env, norm_obs=True, norm_reward=False,
                   clip_obs=10.)

model = PPO2(MlpPolicy, env)
model.learn(total_timesteps=2000)

# Don't forget to save the VecNormalize statistics when saving the agent
log_dir = "/tmp/"
model.save(log_dir + "ppo_reacher")
env.save(os.path.join(log_dir, "vec_normalize.pkl"))

running build_ext


FileNotFoundError: ignored

In [1]:
!git clone https://github.com/openai/mujoco-py
!pip install -e ./mujoco-py

Cloning into 'mujoco-py'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 1949 (delta 0), reused 0 (delta 0), pack-reused 1942
Receiving objects: 100% (1949/1949), 5.57 MiB | 35.67 MiB/s, done.
Resolving deltas: 100% (1189/1189), done.
Obtaining file:///content/mujoco-py
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
Processing /root/.cache/pip/wheels/e3/d8/6e/0d188598f646bff34b79eab04d5203d3649248fbe23253c63f/glfw-1.8.4-cp36-none-any.whl
  Using cached https://files.pythonhosted.org/packages/18/bd/55eb2d6397b9c0e263af9d091ebdb756b15756029b3cededf6461481bc63/fasteners-0.15-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/ac/aa/063eca6a416f397bd99552c534c6d11d57f58f2e94c14780f3bbf818c4cf/monotonic-1.5-py2.py3-none-any.whl
  Running setup.py deve

In [5]:
!ls mujoco-py

Dockerfile  Makefile		pyproject.toml	      scripts
docs	    MANIFEST.in		README.md	      setup.py
examples    mujoco_py		requirements.dev.txt  vendor
LICENSE.md  mujoco_py.egg-info	requirements.txt      xmls
